KPIs and Hypothesis Testing 
- Continuing from previous notebook .Here, first import the data that was prepared for analysis in previous notebook 

In [17]:
import pandas as pd
import numpy as np
import re
from scipy.stats import kruskal  
from scipy.stats import mannwhitneyu
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option("display.width", 200)
London_airbnb=pd.read_csv("London_airbnb_Dashboard_df.csv",low_memory=False)

## ***Pricing KPI'S***

### ***KPI 1: Median Price per Quarter***

**Definition:** Median nightly price of *active listings* in each quarter.  
- *Active listing* = listing with non-missing price.  
- Median is chosen over average to reduce the effect of extreme outliers (luxury listings).  
- This KPI shows the **typical nightly rate** in London Airbnb across time.  


In [20]:
#taking only active listings
active_listing_price=London_airbnb[London_airbnb["price"].notna()]
median_price_per_quarter=(active_listing_price.groupby("quarter")["price"].median().reset_index(name="median_price"))
median_price_per_quarter

,quarter,median_price
0,2024Q3,130.0
1,2024Q4,134.0
2,2025Q1,120.0
3,2025Q2,134.0


### ***Businesss Insight:***
- Prices remain stable around (130–134)pounds, except in Q1 2025 (£120).
- Indicates a seasonal dip in winter demand, with recovery in spring.

### ***Occupancy KPI***

### ***KPI 2 - Occupancy / Availability KPIs***
- compare seasonality in occupancy 
- how much the listed homes are being used, and how this differs across time.

In [3]:
active=London_airbnb[London_airbnb["price"].notna()].copy()
active["occ_rate_30"] = 1 - (active["availability_30"] / 30)
active["occ_rate_30"] = active["occ_rate_30"].clip(0,1)  # make sure between 0–1

occ_per_quarter = (
    active.groupby("quarter")["occ_rate_30"]
    .median()
    .reset_index(name="median_occ_rate")
)

active_counts = (
    active.groupby("quarter")["id"]
    .nunique()
    .reset_index(name="active_listings")
)

result = occ_per_quarter.merge(active_counts, on="quarter")
result

,quarter,median_occ_rate,active_listings
0,2024Q3,0.566667,66194
1,2024Q4,0.533333,65778
2,2025Q1,0.433333,64522
3,2025Q2,0.700000,67144


#### ***Business Insight*** - median_occupancy_rate - Q2(peaked 0.70) > Q3 > Q4 > Q1(demand weakens 0.43 in winter)
#### ***Common Business Insight from results of KPI 1 and KPI 2 is***
- For Quater Q1 both median_occupancy_rate and median_price_drop is least .A seasonal dip indicating deamd softens in winters.

## ***Occupancy and Revenue KPI's***

### ***KPI 3 - Revenue per Quarter KPIs***

In [ ]:
# Step 1: Active listings only
active = London_airbnb[London_airbnb["price"].notna()].copy()

# Step 2: Calculating occupancy approximately
active["occ_rate_30"] = 1 - (active["availability_30"] / 30)
active["occ_rate_30"] = active["occ_rate_30"].clip(0, 1)

# Step 3: Quarterly revenue approximately
active["revenue_proxy_quarter"] = active["price"] * active["occ_rate_30"] * 90

# Step 4: Group by quarter → median revenue aproximately/proxy
revenue_qtr = (
    active.groupby("quarter")["revenue_proxy_quarter"]
    .median()
    .reset_index(name="median_revenue_proxy")
)
revenue_qtr  
#  median_revenue_proxy in pounds

,quarter,median_revenue_proxy
0,2024Q3,5076.0
1,2024Q4,5328.0
2,2025Q1,3780.0
3,2025Q2,6696.0


Observation - Q2(highest revenue) > Q4 > Q3 > Q1(lowest revenue)
- Revenue in Q1 (quater 1,lowest revenue) and Q2 (quanter 2 ,highest revenue) justifies the trend observed in previous 2 KPI's in price and occupancy.

### ***KPI 4: Room Type Contribution to Revenue & Occupancy***
“Which room types (Entire home, Private room, Shared room, Hotel room) generate the most revenue and maintain the highest occupancy?

In [6]:
# Step 1: Active listings
active = London_airbnb[London_airbnb["price"].notna()].copy()

# Step 2: Occupancy proxy
active["occ_rate_30"] = (1 - active["availability_30"]/30).clip(0,1)

# Step 3: Revenue proxy (quarterly)
active["revenue_proxy_quarter"] = active["price"] * active["occ_rate_30"] * 90

# Step 4: Group by room_type
room_summary = (
    active.groupby("room_type")
    .agg(median_occ=("occ_rate_30","median"),
         median_revenue=("revenue_proxy_quarter","median"),
         total_revenue=("revenue_proxy_quarter","sum"))
    .reset_index()
)

# Step 5: Add revenue share
total = room_summary["total_revenue"].sum()
room_summary["revenue_share_%"] = (room_summary["total_revenue"]/total*100).round(1)

room_summary

,room_type,median_occ,median_revenue,total_revenue,revenue_share_%
0,Entire home/apt,0.633333,8175.0,2.009906e+09,87.2
1,Hotel room,0.466667,6888.0,1.013076e+07,0.4
2,Private room,0.433333,2247.0,2.846953e+08,12.3
3,Shared room,0.066667,330.0,1.382812e+06,0.1


### ***Business Insight***
- 87 % of revenue share is generated by Entire Homes(room type) and have high median occupancy too
- Onboarding more Entire Home hosts (especially in under-supplied neighborhoods) is a direct lever for revenue growth.


### ***KPI 5: Property Type Contribution to Revenue & Occupancy***

In [ ]:
# ---------- 1) ACTIVE LISTINGS ONLY ----------
active = London_airbnb[London_airbnb["price"].notna()].copy()
active = active[active["availability_30"].notna()].copy()

# ---------- 2) LISTING-LEVEL METRICS ----------
active["occ_rate_30"] = (1 - active["availability_30"]/30).clip(0,1)
active["revenue_proxy_quarter"] = active["price"] * active["occ_rate_30"] * 90  # ~3 months per quarter

# ---------- 3) GROUP: (room_type × property_type) ----------
combo = (
    active.groupby(["room_type","property_type"])
          .agg(
              listings=("id","nunique"),
              median_occ=("occ_rate_30","median"),
              median_revenue=("revenue_proxy_quarter","median"),
              total_revenue=("revenue_proxy_quarter","sum")
          )
          .reset_index()
)

# Drop very small combos
MIN_LISTINGS = 100
combo = combo[combo["listings"] >= MIN_LISTINGS].copy()

# ---------- 4) REVENUE SHARE & TOP-10 ----------
combo["revenue_share_%"] = (combo["total_revenue"] / combo["total_revenue"].sum() * 100).round(1)
combo_top10 = combo.sort_values("total_revenue", ascending=False).head(10).copy()

# ---------- 5) RESULT ----------
print(combo_top10)


          room_type                property_type  listings  median_occ  median_revenue  total_revenue  revenue_share_%
21  Entire home/apt           Entire rental unit     47110    0.633333          7866.0   1.206121e+09             53.1
17  Entire home/apt                  Entire home      9656    0.566667         10140.0   3.495877e+08             15.4
13  Entire home/apt                 Entire condo     10114    0.666667          8505.0   2.713681e+08             12.0
69     Private room  Private room in rental unit     12021    0.466667          2376.0   1.183302e+08              5.2
22  Entire home/apt    Entire serviced apartment      2238    0.600000          8250.0   8.287598e+07              3.7
61     Private room         Private room in home     11654    0.366667          1683.0   7.488384e+07              3.3
23  Entire home/apt             Entire townhouse       946    0.733333         16711.5   6.220743e+07              2.7
55     Private room        Private room in condo

### ***Business Insights***
- Strategic Supply Growth: Airbnb should prioritize onboarding more entire rental units, homes, and condos .
- Segment Marketing: Private rooms still serve budget travelers — important for inclusivity and occupancy stability.

## ***Host / Supply Dynamics KPIs***

### ***KPI 7 - Does being a Superhost actually influences revenue a lot?***
- A Superhost is a recognition badge Airbnb gives to hosts who consistently deliver exceptional guest experiences.

In [18]:
# 1) Active rows with the inputs required
active = London_airbnb[London_airbnb["price"].notna()].copy()
active = active[active["availability_30"].notna()].copy()

# 2) Making quarter a Period and also keep a string form for easy filtering/printing
active["quarter"] = pd.PeriodIndex(active["quarter"], freq="Q")
active["quarter_str"] = active["quarter"].astype(str)

# Keep exactly the four quarters that make continuous year
wanted_quarters = {"2024Q3","2024Q4","2025Q1","2025Q2"}
one_year = active[active["quarter_str"].isin(wanted_quarters)].copy()

# 3) Occupancy proxy and revenue proxy (quarter)
one_year["occ_rate_30"] = (1 - one_year["availability_30"]/30).clip(0,1)
one_year["revenue_proxy_quarter"] = one_year["price"] * one_year["occ_rate_30"] * 90

# 4) Normalize superhost flag to boolean 
sh_map = {"t": True, "f": False }
one_year["host_is_superhost"] = (one_year["host_is_superhost"].astype(str).str.strip().str.lower().map(sh_map)).fillna(False).astype(bool)

# 5) Aggregate totals across the whole 12-month window
totals = (one_year.groupby("host_is_superhost")["revenue_proxy_quarter"]
          .sum()
          .reindex([False, True])       # ensure both buckets exist
          .fillna(0))

non_sh = totals.loc[False]
sh     = totals.loc[True]
superhost_share_pct = np.nan if (sh+non_sh)==0 else round(sh/(sh+non_sh)*100, 1)

print({
    "non_superhost_revenue": float(non_sh),
    "superhost_revenue": float(sh),
    "superhost_share_%": superhost_share_pct
})

{'non_superhost_revenue': 1675986849.0, 'superhost_revenue': 630127501.5, 'superhost_share_%': np.float64(27.3)}


C:\Users\easntop\AppData\Local\Temp\ipykernel_13504\1414626734.py:19: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  one_year["host_is_superhost"] = (one_year["host_is_superhost"].astype(str).str.strip().str.lower().map(sh_map)).fillna(False).astype(bool)


### ***Business Insight***
- Superhost_share_ contribute 27% of the Airbnb's total revenue because most listings are non-Superhost.
- So being a superhost doesn't contribute largely in Airbnb's revenue.

## ***Trust & Guest Experience KPI's***

### ***KPI 8 -Reviews / Trust KPIs in Review Quality per Quarter***
- Average of cleanliness, location, value scores → median each quarter.
- Business Insight - Assesssing guest experience across each quarter for active listings.


In [ ]:
# Step 1:I Keep only active listings
active = London_airbnb[London_airbnb["price"].notna()].copy()

# Step 2: Compute average quality score for each listing
quality_cols = ["review_scores_cleanliness", "review_scores_location", "review_scores_value"]
active["review_quality"] = active[quality_cols].mean(axis=1, skipna=True)

# Step 3: Group by quarter → median review quality
review_quality_qtr = (
    active.groupby("quarter")["review_quality"]
    .median()
    .reset_index(name="median_review_quality")
)

review_quality_qtr

,quarter,median_review_quality
0,2024Q3,4.756667
1,2024Q4,4.760000
2,2025Q1,4.766667
3,2025Q2,4.763333


### ***Business Insights***
Stability
- guests are happy.
- No major fluctuation across quarters.
- Unlike price and occupancy, quality is not seasonal — it remains stable.
- Airbnb can highlight this in marketing as: “London listings maintain consistently high guest ratings.”

### ***Hypothesis Testing***

***Hypothesis 1***
- ***Null Hypothesis (H0):*** Prices in winter (Q1) are the same as in other quarters.
- ***Alternative Hypothesis (H1):*** Prices in winter (Q1) are significantly lower than in other quarters.
- This is a one-tailed test .
- I'll compare Q1 vs other quarters means 2 groups
- Test used : Mann–Whitney U test (a.k.a. Wilcoxon rank-sum) because Non-parametric, compares two independent samples

In [ ]:
# Active listings only (non-null price)
active = London_airbnb[London_airbnb["price"].notna()].copy()

# Spliting quarters: Q1 vs All other quarters
q1_prices = active.loc[active["quarter"] == "2025Q1", "price"]
non_q1_prices = active.loc[active["quarter"] != "2025Q1", "price"]

# Mann–Whitney U test
stat, p = mannwhitneyu(q1_prices, non_q1_prices, alternative="two-sided")

# Medians for context
median_q1 = q1_prices.median()
median_non_q1 = non_q1_prices.median()

print(f"Mann - Whitney U test: U={stat:.2f}, p={p:.5f}")
print(f"Q1 Median Price = {median_q1:.2f}, Non-Q1 Median Price = {median_non_q1:.2f}")


Mann - Whitney U test: U=5965896631.00, p=0.00000
Q1 Median Price = 120.00, Non-Q1 Median Price = 133.00


***Conclusion*** Since 𝑝 < 0.05 ( is extremely small), reject the null hypothesis.
→ Prices in Q1 are significantly different from other quarters.

***Hypothesis 2***
- ***Null Hypothesis (H0):*** Every pair of quarters has same occupancy rates..
- ***Alternative Hypothesis (H1):*** Every pair of quarters has significantly different occupancy rates.
- Since we are comapring more than 2 groups .
- Test used : Kruskal–Wallis  test .

In [ ]:
active = London_airbnb[London_airbnb["price"].notna()].copy()
active["occ_rate_30"] = (1 - active["availability_30"]/30).clip(0,1)

#Group by Quarter
quarters = {}
for q, subset in active.groupby("quarter"):
    quarters[q] = subset["occ_rate_30"].dropna().values

# (Kruskal–Wallis) 
stat, p = kruskal(*quarters.values())
print(f"Kruskal-Wallis H={stat:.2f}, p={p:.5f}")



Kruskal-Wallis H=4977.21, p=0.00000


***Hypothesis 3***
- ***Null Hypothesis (H₀):*** Median revenue of entire homes/apartments = median revenue of private/shared rooms.
(no significant difference in earnings between property types)
- ***Alternative Hypothesis (H₁):*** Median revenue of entire homes/apartments ≠ median revenue of private/shared rooms.
(significant revenue difference exists)
- Revenue distribution is heavily skewed ( skewness is > 60).
So, we use non-parametric tests:
Mann–Whitney U test if comparing two groups (Entire home vs Private/Shared combined).

In [19]:
# Step 1: For Active listings only
active = London_airbnb[London_airbnb["price"].notna()].copy()
active["occ_rate_30"] = (1 - active["availability_30"] / 30).clip(0,1)
active["revenue_proxy_quarter"] = active["price"] * active["occ_rate_30"] * 90

# Step 2: Define groups
entire = active[active["room_type"]=="Entire home/apt"]["revenue_proxy_quarter"]
non_entire = active[active["room_type"].isin(["Private room","Shared room"])]["revenue_proxy_quarter"]

# Step 3: Mann–Whitney U test
u_stat, p_val = mannwhitneyu(entire, non_entire, alternative="two-sided")

print(f"Mann Whitney U test: U={u_stat:.2f}, p={p_val:.5f}")
print(f"Entire Home Median Revenue = {entire.median():.0f}")
print(f"Private/Shared Median Revenue = {non_entire.median():.0f}")


Mann Whitney U test: U=11941933128.00, p=0.00000
Entire Home Median Revenue = 8175
Private/Shared Median Revenue = 2208


***Conclusion***
- As p(0.0000) is < 0.05: reject null means Entire homes earn significantly higher revenue.
- Medians show the magnitude of difference.
- Entire homes dominate Airbnb revenue, with median revenue of 8175 pounds vs 2208 pounds for private/shared.
- This validates that guests prefer entire properties, and hosts listing entire homes earn substantially more.

***Hypothesis 4***
- ***Null (H₀):*** The distribution of revenue per listing is the same across quarters (no seasonal effect).
- ***Alternative (H₁):*** At least one quarter’s revenue distribution differs significantly (seasonality exists).
- here  >2 groups will be compared (4 quarters: Q3, Q4, Q1, Q2) and revenue is  (skewed)
- Test used is Kruskal–Wallis H-test (non-parametric ANOVA equivalent).


In [14]:
# Using only active listings
active = London_airbnb[London_airbnb["price"].notna()].copy()

# Revenue proxy per quarter
active["occ_rate_30"] = (1 - active["availability_30"]/30).clip(0,1)
active["revenue_proxy_quarter"] = active["price"] * active["occ_rate_30"] * 90

# Split revenue by quarter
groups = [g["revenue_proxy_quarter"].values for _, g in active.groupby("quarter")]

# Kruskal-Wallis test
stat, p = kruskal(*groups)
print(f"Kruskal-Wallis H-test: H={stat:.2f}, p={p:.5f}")

Kruskal-Wallis H-test: H=5198.84, p=0.00000


Since p < 0.05 rejecting null hypothesis H₀ → revenue differs significantly across quarters.

### ***Hypothesis 5*** 
- Null (H₀): Review quality scores are the same across all quarters (no seasonal variation).
- Alternative (H₁): At least one quarter has significantly different review quality scores.
- Review scores are bounded (0–5 scale) and not normally distributed.
- Here for comparing 4 groups (quarters) Kruskal–Wallis test (non-parametric )will be used.

In [15]:
active = London_airbnb[London_airbnb["price"].notna()].copy()

# Compute review quality score
active["review_quality"] = active[["review_scores_cleanliness","review_scores_location","review_scores_value"]].mean(axis=1)

# Group review quality by quarter
groups = [g["review_quality"].dropna().values 
          for _, g in active.groupby("quarter")]

# Kruskal-Wallis test
stat, p = kruskal(*groups)
print(f"Kruskal-Wallis H-test: H={stat:.2f}, p={p:.5f}")

# Also checking medians for context
median_scores = active.groupby("quarter")["review_quality"].median()
print(median_scores)


Kruskal-Wallis H-test: H=35.57, p=0.00000
quarter
2024Q3    4.756667
2024Q4    4.760000
2025Q1    4.766667
2025Q2    4.763333
Name: review_quality, dtype: float64


Here, p < 0.05 -> Rejecting H₀ → There are significant differences 
- Conclusion : Review quality scores are stable — median differences are less than 0.01, which is negligible for customer experience.
- This is a  case where statistical significance not equals business significance. With huge sample sizes, even micro-differences become significant.
